In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn import functional
import torch.optim as optim
import torchvision
from torchvision import transforms, models, datasets
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

In [12]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

model_ft = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
model_ft

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_att

In [13]:
a = model_ft.encoder.layers.encoder_layer_0

In [14]:
a

EncoderBlock(
  (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (dropout): Dropout(p=0.0, inplace=False)
  (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (mlp): MLPBlock(
    (0): Linear(in_features=768, out_features=3072, bias=True)
    (1): GELU(approximate=none)
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=3072, out_features=768, bias=True)
    (4): Dropout(p=0.0, inplace=False)
  )
)

In [20]:
a = torch.rand((3,4,2))

In [21]:
a

tensor([[[4.8423e-01, 2.9871e-01],
         [9.6898e-03, 3.9235e-01],
         [9.6691e-01, 6.4499e-02],
         [8.7230e-01, 1.9455e-01]],

        [[3.2413e-01, 6.5952e-01],
         [9.4883e-01, 3.6410e-01],
         [3.8010e-04, 4.2351e-01],
         [7.5056e-01, 4.1600e-02]],

        [[9.3843e-01, 6.2742e-01],
         [2.2616e-01, 7.7219e-02],
         [5.0713e-01, 3.6874e-01],
         [4.3176e-01, 7.9782e-01]]])

In [18]:
a = a.flatten(2)

In [19]:
a

tensor([[[0.6463, 0.2396],
         [0.1683, 0.6769],
         [0.4322, 0.3274],
         [0.3627, 0.9635]],

        [[0.2609, 0.3613],
         [0.6049, 0.6662],
         [0.1030, 0.4053],
         [0.5458, 0.6204]],

        [[0.6757, 0.5351],
         [0.6667, 0.7874],
         [0.4936, 0.9396],
         [0.5432, 0.4118]]])

In [24]:
a.shape

torch.Size([3, 4, 2])

In [22]:
layer = torch.nn.Linear(2,5)
a = layer(a)

In [22]:
a.expand(3,4,4)

RuntimeError: The expanded size of the tensor (4) must match the existing size (2) at non-singleton dimension 2.  Target sizes: [3, 4, 4].  Tensor sizes: [3, 4, 2]

In [26]:
a

tensor([[[4.8423e-01, 2.9871e-01],
         [9.6898e-03, 3.9235e-01],
         [9.6691e-01, 6.4499e-02],
         [8.7230e-01, 1.9455e-01]],

        [[3.2413e-01, 6.5952e-01],
         [9.4883e-01, 3.6410e-01],
         [3.8010e-04, 4.2351e-01],
         [7.5056e-01, 4.1600e-02]],

        [[9.3843e-01, 6.2742e-01],
         [2.2616e-01, 7.7219e-02],
         [5.0713e-01, 3.6874e-01],
         [4.3176e-01, 7.9782e-01]]])

In [29]:
a[:,0]

tensor([[0.4842, 0.2987],
        [0.3241, 0.6595],
        [0.9384, 0.6274]])

In [18]:
a = torch.rand((5,3,4,4))
a

tensor([[[[0.8635, 0.5002, 0.1439, 0.1032],
          [0.9066, 0.6421, 0.1709, 0.9687],
          [0.7331, 0.8476, 0.8797, 0.2761],
          [0.9224, 0.0965, 0.4081, 0.2961]],

         [[0.0978, 0.3826, 0.2653, 0.9337],
          [0.2563, 0.2839, 0.1476, 0.5878],
          [0.2795, 0.6101, 0.0238, 0.2841],
          [0.5997, 0.1496, 0.3409, 0.2788]],

         [[0.6217, 0.3742, 0.6077, 0.5569],
          [0.4286, 0.0361, 0.5856, 0.7821],
          [0.1564, 0.1503, 0.8068, 0.6646],
          [0.7122, 0.2477, 0.1872, 0.6504]]],


        [[[0.5732, 0.6409, 0.6194, 0.4930],
          [0.8259, 0.1419, 0.4911, 0.1284],
          [0.4421, 0.1239, 0.3021, 0.3016],
          [0.7824, 0.0575, 0.0030, 0.1146]],

         [[0.3798, 0.2149, 0.9618, 0.8657],
          [0.0445, 0.6939, 0.4548, 0.2533],
          [0.1739, 0.0527, 0.1283, 0.0213],
          [0.9015, 0.3030, 0.4513, 0.1097]],

         [[0.7332, 0.1255, 0.9010, 0.7605],
          [0.6820, 0.1031, 0.0120, 0.5042],
          [0.7068, 0

In [3]:
a = torch.mean(a, 2, True)
a

tensor([[[0.5637],
         [0.5764],
         [0.5155],
         [0.4508],
         [0.4349],
         [0.5633],
         [0.5579],
         [0.4748],
         [0.5326],
         [0.4604],
         [0.5781],
         [0.4698],
         [0.4951],
         [0.5483],
         [0.5203],
         [0.3609],
         [0.5339],
         [0.5418],
         [0.5116],
         [0.5037],
         [0.5415],
         [0.5537],
         [0.4525],
         [0.5982],
         [0.4709],
         [0.4805],
         [0.4468],
         [0.5511],
         [0.4794],
         [0.5141],
         [0.5388],
         [0.5129]],

        [[0.4890],
         [0.5022],
         [0.4773],
         [0.4713],
         [0.5456],
         [0.5961],
         [0.4385],
         [0.3769],
         [0.5361],
         [0.4640],
         [0.4213],
         [0.5196],
         [0.5327],
         [0.4769],
         [0.5111],
         [0.5146],
         [0.5663],
         [0.4605],
         [0.4607],
         [0.5077],
         [

In [4]:
a.shape

torch.Size([3, 32, 1])

In [5]:
a = torch.mean(a, 1, True)
a.shape

torch.Size([3, 1, 1])

In [6]:
a

tensor([[[0.5104]],

        [[0.4964]],

        [[0.5023]]])

In [7]:
conv = nn.Conv2d(3,3,1,1,0,bias=True)
bn = nn.BatchNorm2d(3)

In [8]:
a = conv(a)
a.shape

torch.Size([3, 1, 1])

In [9]:
a = bn(a)
a.shape

ValueError: expected 4D input (got 3D input)

In [19]:
ap = nn.AdaptiveAvgPool2d(output_size=1)
a = ap(a)
a.shape

torch.Size([5, 3, 1, 1])

In [20]:
a

tensor([[[[0.5474]],

         [[0.3451]],

         [[0.4730]]],


        [[[0.3776]],

         [[0.3757]],

         [[0.5068]]],


        [[[0.4905]],

         [[0.4440]],

         [[0.4743]]],


        [[[0.5442]],

         [[0.6229]],

         [[0.4282]]],


        [[[0.4891]],

         [[0.5353]],

         [[0.5001]]]])

In [21]:
a = torch.nn.functional.interpolate(a, size=(4,4), mode='bilinear', align_corners=False) 
a.shape

torch.Size([5, 3, 4, 4])

In [22]:
a

tensor([[[[0.5474, 0.5474, 0.5474, 0.5474],
          [0.5474, 0.5474, 0.5474, 0.5474],
          [0.5474, 0.5474, 0.5474, 0.5474],
          [0.5474, 0.5474, 0.5474, 0.5474]],

         [[0.3451, 0.3451, 0.3451, 0.3451],
          [0.3451, 0.3451, 0.3451, 0.3451],
          [0.3451, 0.3451, 0.3451, 0.3451],
          [0.3451, 0.3451, 0.3451, 0.3451]],

         [[0.4730, 0.4730, 0.4730, 0.4730],
          [0.4730, 0.4730, 0.4730, 0.4730],
          [0.4730, 0.4730, 0.4730, 0.4730],
          [0.4730, 0.4730, 0.4730, 0.4730]]],


        [[[0.3776, 0.3776, 0.3776, 0.3776],
          [0.3776, 0.3776, 0.3776, 0.3776],
          [0.3776, 0.3776, 0.3776, 0.3776],
          [0.3776, 0.3776, 0.3776, 0.3776]],

         [[0.3757, 0.3757, 0.3757, 0.3757],
          [0.3757, 0.3757, 0.3757, 0.3757],
          [0.3757, 0.3757, 0.3757, 0.3757],
          [0.3757, 0.3757, 0.3757, 0.3757]],

         [[0.5068, 0.5068, 0.5068, 0.5068],
          [0.5068, 0.5068, 0.5068, 0.5068],
          [0.5068, 0

In [1]:
import torch

In [33]:
conv1 = torch.nn.Conv1d(
            in_channels=3,
            out_channels=10,
            kernel_size=16,
            stride=8,
            padding=8,
            bias=False,
        )

In [26]:
a = torch.randn(3,3,20)

In [20]:
print(a.shape)

torch.Size([3, 20])


In [21]:
a = conv1(a)
print(a.shape)

torch.Size([10, 3])


In [93]:
def _align_num_frames_with_strides(input: torch.Tensor, enc_kernel_size=16, enc_stride=8):
        batch_size, num_channels, num_frames = input.shape
        is_odd = enc_kernel_size % 2
        num_strides = (num_frames - is_odd) // enc_stride
        num_remainings = num_frames - (is_odd + num_strides * enc_stride)
        if num_remainings == 0:
            return input, 0

        num_paddings = enc_stride - num_remainings
        pad = torch.zeros(
            batch_size,
            num_channels,
            num_paddings,
            dtype=input.dtype,
            device=input.device,
        )
        return torch.cat([input, pad], 2), num_paddings

In [27]:
b,c = _align_num_frames_with_strides(a)

In [34]:
print(b.shape)

torch.Size([3, 3, 24])


In [35]:
b = conv1(b)
print(b.shape)

torch.Size([3, 10, 4])


In [36]:
feats = torch.randn(2,1,3)
feats = feats.unsqueeze(1)
print(feats.shape)

torch.Size([2, 1, 1, 3])


In [65]:
a = torch.randn(2,2)
b = torch.randn(2,2)

In [66]:
print(a*b)

tensor([[ 1.3700,  0.0180],
        [ 0.4153, -1.3471]])


In [68]:
from typing import Optional, Tuple
import torch


class ConvBlock(torch.nn.Module):
    """1D Convolutional block.

    Args:
        io_channels (int): The number of input/output channels, <B, Sc>
        hidden_channels (int): The number of channels in the internal layers, <H>.
        kernel_size (int): The convolution kernel size of the middle layer, <P>.
        padding (int): Padding value of the convolution in the middle layer.
        dilation (int, optional): Dilation value of the convolution in the middle layer.
        no_redisual (bool, optional): Disable residual block/output.

    Note:
        This implementation corresponds to the "non-causal" setting in the paper.
    """

    def __init__(
        self,
        io_channels: int,
        hidden_channels: int,
        kernel_size: int,
        padding: int,
        dilation: int = 1,
        no_residual: bool = False,
    ):
        super().__init__()

        self.conv_layers = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=io_channels, out_channels=hidden_channels, kernel_size=1),
            torch.nn.PReLU(),
            torch.nn.GroupNorm(num_groups=1, num_channels=hidden_channels, eps=1e-08),
            torch.nn.Conv1d(
                in_channels=hidden_channels,
                out_channels=hidden_channels,
                kernel_size=kernel_size,
                padding=padding,
                dilation=dilation,
                groups=hidden_channels,
            ),
            torch.nn.PReLU(),
            torch.nn.GroupNorm(num_groups=1, num_channels=hidden_channels, eps=1e-08),
        )

        self.res_out = (
            None
            if no_residual
            else torch.nn.Conv1d(in_channels=hidden_channels, out_channels=io_channels, kernel_size=1)
        )
        self.skip_out = torch.nn.Conv1d(in_channels=hidden_channels, out_channels=io_channels, kernel_size=1)

    def forward(self, input: torch.Tensor) -> Tuple[Optional[torch.Tensor], torch.Tensor]:
        feature = self.conv_layers(input)
        if self.res_out is None:
            residual = None
        else:
            residual = self.res_out(feature)
        skip_out = self.skip_out(feature)
        return residual, skip_out

In [69]:
class MaskGenerator(torch.nn.Module):
    """TCN (Temporal Convolution Network) Separation Module

    Generates masks for separation.

    Args:
        input_dim (int): Input feature dimension, <N>.
        num_sources (int): The number of sources to separate.
        kernel_size (int): The convolution kernel size of conv blocks, <P>.
        num_featrs (int): Input/output feature dimenstion of conv blocks, <B, Sc>.
        num_hidden (int): Intermediate feature dimention of conv blocks, <H>
        num_layers (int): The number of conv blocks in one stack, <X>.
        num_stacks (int): The number of conv block stacks, <R>.
        msk_activate (str): The activation function of the mask output.

    Note:
        This implementation corresponds to the "non-causal" setting in the paper.
    """

    def __init__(
        self,
        input_dim: int,
        num_sources: int,
        kernel_size: int,
        num_feats: int,
        num_hidden: int,
        num_layers: int,
        num_stacks: int,
        msk_activate: str,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.num_sources = num_sources

        self.input_norm = torch.nn.GroupNorm(num_groups=1, num_channels=input_dim, eps=1e-8)
        self.input_conv = torch.nn.Conv1d(in_channels=input_dim, out_channels=num_feats, kernel_size=1)

        self.receptive_field = 0
        self.conv_layers = torch.nn.ModuleList([])
        for s in range(num_stacks):
            for l in range(num_layers):
                multi = 2**l
                self.conv_layers.append(
                    ConvBlock(
                        io_channels=num_feats,
                        hidden_channels=num_hidden,
                        kernel_size=kernel_size,
                        dilation=multi,
                        padding=multi,
                        # The last ConvBlock does not need residual
                        no_residual=(l == (num_layers - 1) and s == (num_stacks - 1)),
                    )
                )
                self.receptive_field += kernel_size if s == 0 and l == 0 else (kernel_size - 1) * multi
        self.output_prelu = torch.nn.PReLU()
        self.output_conv = torch.nn.Conv1d(
            in_channels=num_feats,
            out_channels=input_dim * num_sources,
            kernel_size=1,
        )
        if msk_activate == "sigmoid":
            self.mask_activate = torch.nn.Sigmoid()
        elif msk_activate == "relu":
            self.mask_activate = torch.nn.ReLU()
        else:
            raise ValueError(f"Unsupported activation {msk_activate}")

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """Generate separation mask.

        Args:
            input (torch.Tensor): 3D Tensor with shape [batch, features, frames]

        Returns:
            Tensor: shape [batch, num_sources, features, frames]
        """
        batch_size = input.shape[0]
        feats = self.input_norm(input)
        feats = self.input_conv(feats)
        output = 0.0
        for layer in self.conv_layers:
            residual, skip = layer(feats)
            if residual is not None:  # the last conv layer does not produce residual
                feats = feats + residual
            output = output + skip
        output = self.output_prelu(output)
        output = self.output_conv(output)
        output = self.mask_activate(output)
        return output.view(batch_size, self.num_sources, self.input_dim, -1)


In [82]:
num_sources = 2
        # encoder/decoder parameters
enc_kernel_size = 16
enc_num_feats = 512
        # mask generator parameters
msk_kernel_size = 3
msk_num_feats = 128
msk_num_hidden_feats = 512
msk_num_layers = 8
msk_num_stacks = 3
msk_activate = "sigmoid"

In [87]:
num_sources = num_sources
enc_num_feats = enc_num_feats
enc_kernel_size = enc_kernel_size
enc_stride = enc_kernel_size // 2

In [88]:
print(enc_stride)
# //表示整除（向下取整）

8


In [90]:
encoder = torch.nn.Conv1d(
            in_channels=1,
            out_channels=enc_num_feats,
            kernel_size=enc_kernel_size,
            stride=enc_stride,
            padding=enc_stride,
            bias=False,
        )

In [91]:
mask_generator = MaskGenerator(
            input_dim=enc_num_feats,
            num_sources=num_sources,
            kernel_size=msk_kernel_size,
            num_feats=msk_num_feats,
            num_hidden=msk_num_hidden_feats,
            num_layers=msk_num_layers,
            num_stacks=msk_num_stacks,
            msk_activate=msk_activate,
        )

In [92]:
decoder = torch.nn.ConvTranspose1d(
            in_channels=enc_num_feats,
            out_channels=1,
            kernel_size=enc_kernel_size,
            stride=enc_stride,
            padding=enc_stride,
            bias=False,
        )

In [94]:
input = torch.randn(10,1,1000)

In [96]:
padded, num_pads = _align_num_frames_with_strides(input)
print(padded.shape)
print(num_pads)

torch.Size([10, 1, 1000])
0


In [97]:
batch_size, num_padded_frames = padded.shape[0], padded.shape[2]
print(batch_size)
print(num_padded_frames)

10
1000


In [99]:
feats = encoder(padded)
print(feats.shape)
# 输出为# batchsize, 分频率, M

torch.Size([10, 512, 126])


In [100]:
masked1 = mask_generator(feats)
print(masked1.shape)
# B, S, F, M

torch.Size([10, 2, 512, 126])


In [101]:
feats = feats.unsqueeze(1)
print(feats.shape)

In [102]:
masked = masked1 * feats  # *为对应元素相乘，@为矩阵乘，此处为对应元素乘，F和M的维度必须一样
print(masked.shape)
# B, S, F, M

torch.Size([10, 2, 512, 126])


In [103]:
masked = masked.view(batch_size * num_sources, enc_num_feats, -1)
print(masked.shape)
# B*S, F, M

torch.Size([20, 512, 126])


In [104]:
decoded = decoder(masked)
print(decoded.shape)
# B*S, 1, L'

torch.Size([20, 1, 1000])


In [105]:
output = decoded.view(batch_size, num_sources, num_padded_frames)
print(output.shape)
# B, S, L'

torch.Size([10, 2, 1000])


In [106]:
if num_pads > 0:
    output = output[..., :-num_pads]
# B, S, L
# 这一步将填充的pads去掉

太神奇啦，居然能将输入和输出的维度对上！！！

In [1]:
"""
时间：2022/10/19
作者：余俊锐
审查：余俊锐
版权归属：广工计算机研学会学术部
使用说明： 在方法上创建工作路径，命名按第几期进行命名，创建文件夹‘报名名单’和‘结果’，分别
        ‘第x期’中放置需要添加的人员名单，列名【姓名，学号】
        ‘报名名单’中放置报名工具生成的名单，需要将标题删除后再使用
        ‘结果’为最后生成的数据，点击打开后，重新保存，因为有可能出现格式错误，重新保存可以解决
        方法增加了学号校验，学院校验
"""
import os

import numpy as np
import pandas as pd
import re

if __name__ == "__main__":
    """
    该文件用于报名工具名单的人员处理
    1、创建对应期数的文件夹作为工作路径，修改工作路径
    2、在对应工作路径下创建 报名名单 和 结果 文件夹
    3、在报名名单下添加 报名文件
    4、如果有额外添加的人员，新建excel文件放置在work_path路径下，列名为【姓名，学号】
    5、筛选结果人数为len_people
    6、使用前先将非本学院的人从报名工具当中去除
    """
    # -------------------- 参数设置------------------------
    '''工作路径'''
    work_path = r"C:\Users\54469\Desktop"
    '''报名名单的相对路径，将读取该路径下的文件作为报名的文件的路径'''
    sign_path = '{}/报名名单'.format(work_path)
    '''导出会议名单相对路径'''
    out_path = '{}/结果'.format(work_path)
    '''截取人数'''
    len_people = 150

    # -------------------处理报名名单----------------------
    files = []
    for index, file in enumerate(os.listdir(sign_path)):  #
        if file.endswith('.xls') and not file.startswith('公示名单'):
            files.append(file)
        else:
            pass

    add_files = []
    for index, file in enumerate(os.listdir(work_path)):  #
        if file.endswith('.xls') and not file.startswith('公示名单'):
            add_files.append(file)
        else:
            pass

    if len(files) == 0:
        print('请在指定位置添加报名名单')
    print('报名files: {}'.format(files))
    '''xls文件处理'''
    for file in files:
        file_name = file.split("-")  # 选择file_name 作为最后的文档名拼接
        file = '{}/{}'.format(sign_path, file)
        write = pd.ExcelWriter('{}/{}-报名公示.xlsx'.format(out_path, file_name[0]))
        sign_up_list_data = pd.read_excel(file, engine='xlrd', dtype='object')
        print(sign_up_list_data['姓名'].notnull())
        null_pd = pd.DataFrame()
        null_pd.to_excel(write, index=False, sheet_name="报名成功名单")
        null_pd.to_excel(write, index=False, sheet_name="筛选名单")
        sign_up_list_data.to_excel(write, index=False, sheet_name="原始数据")
        sign_up_list_data['随机数'] = 0
        sign_up_list_data = sign_up_list_data[sign_up_list_data['姓名'].notnull()]
        sign_up_list_data['学号'] = sign_up_list_data['学号'].astype(str)
        sign_up_list_data['学号'] = sign_up_list_data['学号'].str.strip()
        sign_up_list_data = sign_up_list_data[sign_up_list_data['学号'].str.startswith('2112')]
        # sign_up_list_data = sign_up_list_data[sign_up_list_data['学院'].str.contains('计')]
        sign_up_list_data['随机数'] = np.random.rand(len(sign_up_list_data['姓名']),1)
        sign_up_list_data = sign_up_list_data.sort_values('随机数', ascending=False)
        # sign_up_list_data.to_excel(write, columns=['姓名', '学号', '随机数'], index=False, sheet_name='筛选名单')
        sign_up_list_data.to_excel(write, columns=['唯一编号', '昵称', '姓名', '学号', '学院', '随机数'], index=False, sheet_name="筛选名单")
        new_data = sign_up_list_data[0:len_people]
        new_data = new_data.sort_values('学号')
        # ----------添加额外人员----------
        if len(add_files) != 0:
            for add_file in add_files:
                add_file = '{}/{}'.format(work_path, add_file)
                add_data_pd = pd.read_excel(add_file, engine='xlrd', dtype='object')
                add_data_pd['学号'] = add_data_pd['学号'].astype(str)
                add_data_pd['学号'] = add_data_pd['学号'].str.strip()
                new_data = new_data.append(add_data_pd)
                new_data['学院'] = '计算机学院'
        # ----------添加额外人员----------
        new_data.to_excel(write, columns=['学号', '姓名', '学院', '随机数'], index=False, sheet_name="报名成功名单")
        write.save()

报名files: ['CCF广东工业大学学生分会换届大会暨”智能影像分析技术与案例研讨“学术专题讲座-.xls']
0       True
1       True
2       True
3       True
4       True
       ...  
313     True
314    False
315    False
316    False
317    False
Name: 姓名, Length: 318, dtype: bool


C:\Users\54469\AppData\Local\Temp\ipykernel_13352\3487757190.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(add_data_pd)


简单的文件处理

In [2]:
import os



datadir = r"E:\myfile\2022-2023\各种奇怪的代码\pytorch_tutorial\028_034：图像识别核心模块实战解读\卷积网络实战\flower_data"
a = 0

for i in os.listdir(datadir):
    sdir = os.path.join(datadir, i)
    for j in os.listdir(sdir):
        tdir = os.path.join(sdir, j)
        a += len(os.listdir(tdir))
    print(a)
print(a)



27
38
35
49
36
68
73
38
44
38
36
60
65
38
49
46
34
47
72
35
34
33
36
55
62
36
61
48
36
31
28
33
62
92
44
33
44
54
97
49
100
73
33
157
61
57
38
54
73
206
67
70
47
56
92
50
86
56
35
85
36
48
42
42
88
51
36
43
46
33
51
64
77
147
142
95
83
205
112
34
70
82
135
82
104
66
48
48
51
116
153
41
66
59
53
34
132
101
72
54
68
50
8
4
6
5
6
10
5
5
1
7
2
16
11
4
6
7
4
8
12
5
2
3
1
5
7
2
10
2
3
7
7
4
6
8
4
3
6
5
16
6
14
9
4
18
3
9
8
7
11
28
10
9
10
8
9
6
14
4
1
14
6
3
8
5
7
6
2
8
5
1
7
5
8
19
15
12
20
21
11
4
5
12
18
13
13
10
5
5
6
25
16
3
2
9
2
6
14
13
10
7
10
6
7370


In [44]:
alpha=0.25
max_gamma=3
num_classes=2
size_average=True
epoch=200
split=4

In [45]:
gamma_split_span = epoch / split
gamma_span = max_gamma / (split-1)

In [46]:
gamma_split_span

50.0

In [47]:
gamma_span

1.0

In [15]:
gamma_split = list(range(0, epoch, gamma_split_span))
gamma_split = [*map(lambda x:x+gamma_split_span, gamma_split)]
gamma_split

[50, 100, 150, 200]

In [56]:
gamma = []
gamma_split = []
n = 0
m = 0
for i in range(split):
    m += gamma_split_span
    n += gamma_span
    gamma.append(n)
    gamma_split.append(m)
    
gamma

[1.0, 2.0, 3.0, 4.0]

In [57]:
gamma_split

[50.0, 100.0, 150.0, 200.0]

In [53]:
for i,n in enumerate(gamma_split):
    print(i)
    print(n)

0
50.0
1
100.0
2
150.0
3
200.0


空间注意力

In [11]:
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, padding=7 // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        # 压缩通道提取空间信息
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        # 经过卷积提取空间注意力权重
        x = torch.cat([max_out, avg_out], dim=1)
        out = self.conv1(x)
        # 输出非负
        out = self.sigmoid(out)
        return out

In [12]:
a = torch.randn(2,3,5,5)
a

tensor([[[[ 1.4608,  0.6083,  0.6780,  1.2978, -0.5806],
          [ 0.8322, -0.3913,  0.2327,  1.1116,  0.0886],
          [ 2.5856,  1.0699,  0.6909,  1.8917,  1.1411],
          [-0.4727,  0.8836,  0.0776, -0.0390,  1.3603],
          [-1.7871, -0.5773, -0.3445,  0.4017, -1.2424]],

         [[-0.5791, -1.1577,  0.4284,  1.5596, -1.1020],
          [ 0.5986, -1.8203,  0.5768,  0.2990, -1.4570],
          [ 0.2605, -1.1725,  0.8634, -1.1953, -0.7474],
          [-0.9153, -0.4563, -0.6334,  0.3975, -1.2407],
          [-0.8747, -2.3684,  2.5104, -0.8198, -0.9196]],

         [[ 0.4632,  0.0409,  0.7671, -1.0282, -1.6957],
          [ 1.2146, -1.5956, -0.7351,  0.4318, -1.5435],
          [ 2.2750, -0.3302, -0.7520, -0.3099, -0.3498],
          [ 1.6909,  0.6941, -1.3569,  2.1984,  0.1538],
          [-0.4788,  2.5777, -0.5045, -1.5407, -0.2768]]],


        [[[-1.6690,  0.7950,  0.5913, -0.5904, -1.8864],
          [-0.0899,  0.2393,  1.4223, -2.4186, -0.7062],
          [ 1.4658, -1.

In [13]:
sp = SpatialAttention()
sp

SpatialAttention(
  (conv1): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (sigmoid): Sigmoid()
)

In [14]:
b = sp(a)
b

tensor([[[[0.4239, 0.4664, 0.3596, 0.3945, 0.5226],
          [0.2871, 0.3955, 0.3793, 0.3147, 0.5999],
          [0.3946, 0.4530, 0.4554, 0.3876, 0.4538],
          [0.3273, 0.4378, 0.5477, 0.4588, 0.4210],
          [0.4779, 0.4016, 0.4806, 0.3932, 0.4439]]],


        [[[0.4771, 0.5512, 0.4085, 0.4787, 0.5531],
          [0.4732, 0.4148, 0.5022, 0.3442, 0.4558],
          [0.3887, 0.5473, 0.4845, 0.5381, 0.3776],
          [0.3570, 0.4914, 0.4468, 0.5896, 0.4642],
          [0.4307, 0.4269, 0.5480, 0.4001, 0.3856]]]],
       grad_fn=<SigmoidBackward0>)

In [15]:
b.size()

torch.Size([2, 1, 5, 5])

In [16]:
c = a * b
c.size()

torch.Size([2, 3, 5, 5])

In [20]:
c

tensor([[[[ 0.0818,  0.6467, -0.5399, -0.1180, -0.7981],
          [-0.3033,  0.6890,  0.7490,  0.6174,  0.2510],
          [ 0.0907, -0.5988,  0.1723, -0.2069,  0.1348],
          [ 0.8769,  0.3614,  0.7397, -0.3258,  0.0577],
          [-0.6559, -0.5409, -0.6703, -0.0847,  0.4660]],

         [[-0.2667,  0.0308, -0.0726, -0.3346, -0.0209],
          [ 0.4650,  0.8673,  0.4554,  0.2898,  0.2319],
          [ 1.0282, -0.2046,  0.4887,  0.0196, -0.5363],
          [-0.0701, -0.2793,  0.6749, -0.7656,  0.2678],
          [-0.3965,  0.5835, -0.4100,  0.0703,  0.0817]],

         [[-0.3808,  0.1436, -0.7972,  0.3561, -0.1837],
          [ 0.4568,  0.2451, -0.0555,  0.6382,  0.1721],
          [ 0.0077, -0.1413, -0.3243, -0.2507, -0.5343],
          [ 0.0936,  0.0061,  0.2226, -0.5326, -1.1568],
          [-0.6950,  0.3353, -0.0165,  0.0246,  0.7882]]],


        [[[ 0.6962,  0.5488, -0.6768, -0.2350,  0.4656],
          [-1.2748,  0.7762, -0.0516, -0.7424, -0.1041],
          [ 0.5736, -0.

In [22]:
conv1 = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, padding=7 // 2, bias=False)
sigmoid = nn.Sigmoid()

In [23]:
max_out, _ = torch.max(a, dim=1, keepdim=True)
max_out.size()

torch.Size([2, 1, 5, 5])

In [27]:
avg_out = torch.mean(a, dim=1, keepdim=True)
avg_out.size()

torch.Size([2, 1, 5, 5])

In [25]:
# 经过卷积提取空间注意力权重
x = torch.cat([max_out, avg_out], dim=1)
x.size()

torch.Size([2, 2, 5, 5])

In [30]:
out = conv1(x)
out.size()

torch.Size([2, 1, 5, 5])

In [31]:
out = sigmoid(out)
out.size()

torch.Size([2, 1, 5, 5])

In [33]:
ap = torch.nn.AdaptiveAvgPool2d(1)
ap

AdaptiveAvgPool2d(output_size=1)

In [34]:
b = ap(a)
b.size()

torch.Size([2, 3, 1, 1])

In [35]:
b = torch.mean(a, dim=1, keepdim=True)
b.size()

torch.Size([2, 1, 5, 5])

In [43]:
class SpatialAttention(torch.nn.Module):
    # 空间注意力
    def __init__(
            self,
            activation = torch.nn.ReLU,
            scale_activation = torch.nn.Sigmoid,
    ) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=7, padding=7 // 2, bias=False)
        self.scale_activation = scale_activation()
        self.activation = activation()

    def _scale(self, x):
        scale = torch.mean(x, dim=1, keepdim=True)  # 压缩通道，B,1,W,H
        scale = self.activation(scale)              # 过激活函数，Swish
        scale = self.conv1(scale)                   # 7*7卷积
        return self.scale_activation(scale)         # 过S函数

    def forward(self, x):
        scale = self._scale(x)
        return scale * x

In [44]:
sp = SpatialAttention()
sp

SpatialAttention(
  (conv1): Conv2d(1, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (scale_activation): Sigmoid()
  (activation): ReLU()
)

In [45]:
a = torch.randn(2,3,5,5)
a

tensor([[[[ 0.5286, -0.4347,  0.5691,  0.1707, -0.3151],
          [ 0.4654,  0.5342,  0.7557,  0.2657,  2.0248],
          [-0.6644, -0.2342,  0.4695,  0.8974,  0.2327],
          [-0.9005, -1.3277,  0.8622, -2.1104, -0.9401],
          [ 0.9488, -1.0526,  0.5012, -1.0522,  2.6167]],

         [[-1.6506,  1.3244,  0.0674, -1.0445, -1.2324],
          [-0.0399, -0.5837, -0.6501,  0.9812,  0.4808],
          [-0.3467,  1.4414, -1.1650,  0.4514, -1.2616],
          [ 0.6378,  1.9424, -1.7958,  0.8658,  0.4773],
          [ 0.6408, -0.1609, -0.2765, -0.2740,  0.8477]],

         [[ 0.3064, -1.7544, -0.5060, -0.6738,  1.8561],
          [-0.1998,  0.6680,  1.2047,  1.3519, -0.7833],
          [-0.6256, -1.5222, -0.1914, -0.7464, -0.0217],
          [ 0.5127, -0.6052,  0.0545,  1.4577, -1.7289],
          [-1.2410, -0.0642, -0.7631, -0.9726,  0.4959]]],


        [[[ 0.5864, -0.3693,  0.1426,  1.4548, -2.0316],
          [ 0.7912,  0.5848,  1.0839,  1.7743, -0.7303],
          [ 0.8964, -1.

In [37]:
scale = torch.mean(a, dim=1, keepdim=True)
scale.size()

torch.Size([2, 1, 5, 5])

In [38]:
relu = torch.nn.ReLU()
scale = relu(scale)
scale.size()

torch.Size([2, 1, 5, 5])

In [39]:
conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=7, padding=7 // 2, bias=False)
scale = conv1(scale)
scale.size()

torch.Size([2, 1, 5, 5])

In [40]:
s = torch.nn.Sigmoid()
scale = s(scale)
scale.size()

torch.Size([2, 1, 5, 5])

In [41]:
a.size()

torch.Size([2, 3, 5, 5])

In [42]:
out = scale * a
out.size()

torch.Size([2, 3, 5, 5])

In [46]:
out = sp(a)
out.size()

torch.Size([2, 3, 5, 5])

三层的空洞卷积

In [77]:
conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, dilation=1)
conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=2, dilation=2)
conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=4, dilation=4)
conv4 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=8, dilation=8)

In [78]:
a = torch.randn(2,3,5,5)
a.size()

torch.Size([2, 3, 5, 5])

In [79]:
b = conv1(a)
b.size()

torch.Size([2, 3, 5, 5])

In [80]:
c = conv2(b)
c.size()

torch.Size([2, 3, 5, 5])

In [81]:
d = conv3(c)
d.size()

torch.Size([2, 3, 5, 5])

In [82]:
e = conv4(d)
e.size()

torch.Size([2, 3, 5, 5])

In [83]:
out = torch.cat([b, c, d, e], dim=1)
out.size()

torch.Size([2, 12, 5, 5])

测试模型集成

In [2]:
conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False)
conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False)

In [8]:
conv1.parameters()

<generator object Module.parameters at 0x000001AD3251C2E0>

In [10]:
for i in conv1.parameters():
    a = i
print(a)

Parameter containing:
tensor([[[[ 0.0613,  0.0534, -0.0373],
          [-0.1193,  0.1856, -0.0455],
          [-0.0191,  0.1048, -0.1513]],

         [[-0.0080, -0.1112,  0.0291],
          [ 0.1007,  0.0685,  0.0521],
          [-0.1563,  0.1774, -0.1910]],

         [[-0.1268, -0.0416,  0.0750],
          [ 0.1187, -0.0324, -0.0876],
          [ 0.1622,  0.1158,  0.0803]]],


        [[[-0.1065,  0.1076,  0.0106],
          [-0.0357, -0.1750, -0.1482],
          [-0.0489, -0.0109,  0.0354]],

         [[-0.1038, -0.0647,  0.0012],
          [-0.1069, -0.1338, -0.1513],
          [ 0.1417, -0.0964, -0.1218]],

         [[ 0.1808,  0.0858,  0.0669],
          [ 0.1528, -0.0935, -0.1912],
          [ 0.1616, -0.0246,  0.1493]]],


        [[[ 0.1852,  0.0074,  0.0914],
          [ 0.0126, -0.1117,  0.0093],
          [ 0.1859, -0.1363,  0.1835]],

         [[ 0.1081, -0.1156,  0.1336],
          [ 0.0487,  0.1908,  0.1723],
          [-0.1201,  0.0602, -0.1413]],

         [[-0.0092, -0

In [11]:
a.size()

torch.Size([3, 3, 3, 3])

In [12]:
for i in conv2.parameters():
    b = i
b.size()

torch.Size([3, 3, 3, 3])

In [14]:
b

Parameter containing:
tensor([[[[ 0.0949, -0.0326,  0.0070],
          [ 0.1715, -0.0771,  0.1806],
          [ 0.0424,  0.1182,  0.1553]],

         [[-0.0139,  0.0150,  0.1008],
          [-0.1282, -0.0725, -0.1651],
          [-0.1797,  0.1176,  0.1500]],

         [[ 0.0359, -0.1809, -0.0233],
          [ 0.0291, -0.1108, -0.1709],
          [ 0.0474,  0.1575, -0.0795]]],


        [[[-0.0592, -0.1730, -0.1429],
          [ 0.1810, -0.0887, -0.0724],
          [ 0.1291,  0.1172,  0.1442]],

         [[ 0.0321,  0.0045, -0.1718],
          [-0.0100,  0.0268, -0.1332],
          [ 0.1294,  0.0101, -0.1216]],

         [[ 0.1019,  0.0823, -0.0273],
          [ 0.0560,  0.1742,  0.1505],
          [-0.1627,  0.0174,  0.1565]]],


        [[[ 0.1098,  0.0660,  0.1221],
          [ 0.1832,  0.1307, -0.0482],
          [ 0.0217, -0.0561,  0.0732]],

         [[-0.1410,  0.0575,  0.1152],
          [ 0.0413, -0.0506, -0.1584],
          [-0.0271,  0.0445, -0.0884]],

         [[ 0.1182,  0

In [27]:
from torch.autograd import Variable

c = (a + b) / 2
c = Variable(c)
c.requires_grad=True
c

tensor([[[[ 7.8101e-02,  1.0412e-02, -1.5171e-02],
          [ 2.6091e-02,  5.4220e-02,  6.7543e-02],
          [ 1.1616e-02,  1.1151e-01,  2.0059e-03]],

         [[-1.0941e-02, -4.8102e-02,  6.4917e-02],
          [-1.3779e-02, -2.0113e-03, -5.6515e-02],
          [-1.6798e-01,  1.4748e-01, -2.0509e-02]],

         [[-4.5459e-02, -1.1121e-01,  2.5882e-02],
          [ 7.3860e-02, -7.1595e-02, -1.2922e-01],
          [ 1.0478e-01,  1.3667e-01,  3.8619e-04]]],


        [[[-8.2822e-02, -3.2692e-02, -6.6125e-02],
          [ 7.2682e-02, -1.3185e-01, -1.1033e-01],
          [ 4.0104e-02,  5.3166e-02,  8.9800e-02]],

         [[-3.5824e-02, -3.0077e-02, -8.5321e-02],
          [-5.8438e-02, -5.3528e-02, -1.4223e-01],
          [ 1.3552e-01, -4.3111e-02, -1.2169e-01]],

         [[ 1.4135e-01,  8.4033e-02,  1.9830e-02],
          [ 1.0438e-01,  4.0369e-02, -2.0362e-02],
          [-5.6192e-04, -3.6431e-03,  1.5290e-01]]],


        [[[ 1.4749e-01,  3.6720e-02,  1.0673e-01],
          [ 9.7

In [22]:
x = torch.randn(2,3,5,5)
x.size()

torch.Size([2, 3, 5, 5])

In [23]:
bias = torch.zeros(3)
bias

tensor([0., 0., 0.])

In [24]:
out = functional.conv2d(x, c, bias, stride=1, padding=1)
out.size()

torch.Size([2, 3, 5, 5])

测试sequencial

In [2]:
layers = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False))
layers

Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [47]:
model = torchvision.models.efficientnet_v2_s()

In [50]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [55]:
len(model.features)

8

In [56]:
model.features[2]

Sequential(
  (0): FusedMBConv(
    (block): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(24, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Conv2dNormActivation(
        (0): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stochastic_depth): StochasticDepth(p=0.01, mode=row)
  )
  (1): FusedMBConv(
    (block): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(48, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Conv2dNormActivation(
        (0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [57]:
len(model.features[2])

4

In [59]:
model.features[2][1]

FusedMBConv(
  (block): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(48, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Conv2dNormActivation(
      (0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (stochastic_depth): StochasticDepth(p=0.015000000000000003, mode=row)
)

In [4]:
layers1 = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False))
layers2 = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False))
layers2

Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [5]:
layers1.state_dict()

OrderedDict([('0.weight',
              tensor([[[[ 0.0804,  0.1209, -0.0205],
                        [ 0.0005,  0.0665,  0.1233],
                        [-0.0352, -0.0695,  0.0064]],
              
                       [[ 0.1561, -0.0747,  0.1415],
                        [-0.1410,  0.0719, -0.0140],
                        [-0.0071, -0.0655, -0.0447]],
              
                       [[ 0.0679,  0.1416,  0.1870],
                        [-0.1270, -0.1289,  0.1186],
                        [ 0.0553, -0.1193,  0.1104]]],
              
              
                      [[[-0.0786, -0.0536, -0.1099],
                        [ 0.0004,  0.1109,  0.1834],
                        [-0.0327,  0.0480, -0.1861]],
              
                       [[-0.0844, -0.0950, -0.1664],
                        [ 0.0822, -0.0645,  0.1251],
                        [-0.0187,  0.0977,  0.0022]],
              
                       [[ 0.1252,  0.0265,  0.1829],
                        [-0.12

In [6]:
layers2.state_dict()

OrderedDict([('0.weight',
              tensor([[[[-0.0242, -0.1919, -0.1326],
                        [-0.1720, -0.1863, -0.1205],
                        [-0.1607,  0.0020,  0.1276]],
              
                       [[ 0.1180, -0.0572, -0.0044],
                        [ 0.0319, -0.1746,  0.1193],
                        [-0.0705,  0.1544,  0.1866]],
              
                       [[-0.0490,  0.0407,  0.1921],
                        [ 0.1858,  0.1481,  0.1553],
                        [-0.1286, -0.0533,  0.1309]]],
              
              
                      [[[ 0.0246,  0.0983, -0.0317],
                        [-0.1224,  0.1273, -0.1132],
                        [ 0.0494, -0.0639, -0.1674]],
              
                       [[ 0.0634, -0.0706,  0.1228],
                        [-0.0062,  0.0901, -0.1577],
                        [ 0.1310,  0.0853, -0.0079]],
              
                       [[ 0.1724, -0.0497, -0.1303],
                        [ 0.09

In [6]:
a = layers1.state_dict()
b = layers2.state_dict()
c = layers.state_dict()

In [7]:
for key, _ in c.items():
    c[key] = (a[key] + b[key]) / 2
c

OrderedDict([('0.weight',
              tensor([[[[ 0.0261, -0.1427,  0.0538],
                        [-0.0548,  0.1374,  0.0751],
                        [-0.0847,  0.1487,  0.1235]],
              
                       [[-0.0438, -0.1285,  0.0451],
                        [ 0.1478, -0.0203, -0.0695],
                        [ 0.0942, -0.0340,  0.1376]],
              
                       [[-0.0518, -0.0011, -0.0732],
                        [-0.0415, -0.1470, -0.0437],
                        [ 0.1803,  0.0457,  0.1090]]],
              
              
                      [[[ 0.0472,  0.0206, -0.0253],
                        [-0.0961,  0.0096,  0.0516],
                        [ 0.0164,  0.0795,  0.0694]],
              
                       [[-0.1107, -0.0778, -0.0783],
                        [ 0.0272, -0.0560,  0.0170],
                        [-0.1020,  0.0167,  0.0248]],
              
                       [[ 0.0937, -0.1231,  0.0391],
                        [ 0.03

In [11]:
print(de)
print(df)

tensor([[[[ 0.0428, -0.1024,  0.0283],
          [-0.1318, -0.1535,  0.1572],
          [-0.1419,  0.1229, -0.1640]],

         [[ 0.1426,  0.0403, -0.1799],
          [ 0.0687, -0.1622, -0.0862],
          [-0.1168,  0.1757, -0.0870]],

         [[-0.0113, -0.0581, -0.0875],
          [ 0.0360, -0.0316, -0.0644],
          [-0.0563, -0.0620, -0.1304]]],


        [[[-0.0606, -0.0245,  0.0080],
          [ 0.1349,  0.1413, -0.0616],
          [-0.0020, -0.1537, -0.1713]],

         [[-0.1287,  0.0627,  0.1010],
          [ 0.0280,  0.1441,  0.0895],
          [ 0.1651, -0.1387,  0.0508]],

         [[-0.0072,  0.0499,  0.1699],
          [-0.0390,  0.1430,  0.0649],
          [-0.0182, -0.0872,  0.1310]]],


        [[[ 0.1911,  0.0292,  0.0816],
          [ 0.1842,  0.1207,  0.1178],
          [ 0.0557,  0.1149,  0.0984]],

         [[ 0.1396, -0.0530, -0.0011],
          [ 0.1397, -0.1188, -0.1339],
          [-0.1699, -0.0168, -0.0249]],

         [[-0.0986,  0.0848,  0.1193],
     

In [12]:
b[df]

tensor([[[[ 0.1479, -0.1750,  0.0385],
          [ 0.0264,  0.0729,  0.0716],
          [ 0.1330,  0.0751,  0.0067]],

         [[-0.1411,  0.0937, -0.0955],
          [ 0.0413, -0.0771,  0.0984],
          [-0.1882, -0.0973,  0.1464]],

         [[ 0.0297, -0.0625,  0.0100],
          [ 0.1075, -0.1678, -0.1001],
          [ 0.0459, -0.0566, -0.0601]]],


        [[[ 0.0860, -0.0543, -0.0205],
          [-0.0103,  0.0547,  0.1426],
          [ 0.1833, -0.0577,  0.0794]],

         [[ 0.0450,  0.1090,  0.0671],
          [ 0.0186, -0.1879,  0.0630],
          [ 0.1614,  0.0275, -0.0953]],

         [[-0.0898,  0.1174,  0.1518],
          [ 0.0523,  0.0026,  0.1202],
          [-0.0429, -0.0648, -0.0108]]],


        [[[-0.0502, -0.1234,  0.1208],
          [ 0.0424,  0.0588,  0.0948],
          [-0.0841, -0.1444,  0.0283]],

         [[-0.0936,  0.0960,  0.0175],
          [-0.0388,  0.0501,  0.1359],
          [ 0.1061,  0.0464,  0.0736]],

         [[ 0.1862, -0.0310,  0.1073],
     

In [15]:
c[df] = (a[df] + b[df]) / 2
c[df]

tensor([[[[ 0.0953, -0.1387,  0.0334],
          [-0.0527, -0.0403,  0.1144],
          [-0.0045,  0.0990, -0.0786]],

         [[ 0.0007,  0.0670, -0.1377],
          [ 0.0550, -0.1196,  0.0061],
          [-0.1525,  0.0392,  0.0297]],

         [[ 0.0092, -0.0603, -0.0388],
          [ 0.0717, -0.0997, -0.0823],
          [-0.0052, -0.0593, -0.0952]]],


        [[[ 0.0127, -0.0394, -0.0062],
          [ 0.0623,  0.0980,  0.0405],
          [ 0.0907, -0.1057, -0.0459]],

         [[-0.0418,  0.0858,  0.0841],
          [ 0.0233, -0.0219,  0.0762],
          [ 0.1633, -0.0556, -0.0223]],

         [[-0.0485,  0.0837,  0.1609],
          [ 0.0067,  0.0728,  0.0926],
          [-0.0305, -0.0760,  0.0601]]],


        [[[ 0.0705, -0.0471,  0.1012],
          [ 0.1133,  0.0898,  0.1063],
          [-0.0142, -0.0147,  0.0634]],

         [[ 0.0230,  0.0215,  0.0082],
          [ 0.0505, -0.0344,  0.0010],
          [-0.0319,  0.0148,  0.0243]],

         [[ 0.0438,  0.0269,  0.1133],
     

In [27]:
layers = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.AdaptiveAvgPool2d(1))
linear = nn.Linear(3,2,bias=False)

In [30]:
layers1 = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.AdaptiveAvgPool2d(1))
layers2 = torch.nn.Sequential(nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.BatchNorm2d(3),
                            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, bias=False),
                            nn.AdaptiveAvgPool2d(1))

In [32]:
a = layers1.state_dict()
b = layers2.state_dict()
c = layers.state_dict()

for key, _ in c.items():
    c[key] = (a[key] + b[key]) / 2

layers1.load_state_dict(a)
layers2.load_state_dict(b)
layers.load_state_dict(c)

<All keys matched successfully>

In [33]:
x = torch.randn(2,3,5,5)
x.size()

torch.Size([2, 3, 5, 5])

In [34]:
out1 = layers(x)
out1 = out1.view(2,-1)
linear(out1)

tensor([[-0.0296,  0.0398],
        [ 0.0060, -0.0063]], grad_fn=<MmBackward0>)

In [35]:
out1 = layers1(x)
out1 = out1.view(2,-1)
linear(out1)

tensor([[-0.0149,  0.0388],
        [ 0.0115, -0.0212]], grad_fn=<MmBackward0>)

In [36]:
out1 = layers2(x)
out1 = out1.view(2,-1)
linear(out1)

tensor([[ 0.0989,  0.0115],
        [-0.0474,  0.0109]], grad_fn=<MmBackward0>)

测试模型集成代码

In [17]:
# x = torch.randn(3,2)
# _, a = torch.max(x, dim=1)
# print(a)

# x = torch.randn(3,2)
# _, b = torch.max(x, dim=1)
# print(b)


# x = torch.randn(3,2)
# _, c = torch.max(x, dim=1)
# print(c)

# d = [a,b,c]
# d = torch.stack(d,dim=1)
# d = d.squeeze(1)
# print(d.shape)

x1 = torch.randn(3,2)
x2 = torch.randn(3,2)
x3 = torch.randn(3,2)
d = [x1,x2,x3]
d = torch.cat(d,1)
print(d.shape)



torch.Size([3, 6])


In [18]:
sum(c[0,:]==1).item()

NameError: name 'c' is not defined

In [153]:
c_ = []
for r in range(c.size(0)):
    num_0 = sum(c[r,:]==0)
    num_1 = sum(c[r,:]==1)
    a = torch.randn(2)
    a[0] = num_0
    a[1] = num_1
    c_.append(a)
    
print(c_)

[tensor([1., 2.]), tensor([1., 2.]), tensor([2., 1.])]


In [154]:
d = torch.stack(c_,dim=0)
print(d)

tensor([[1., 2.],
        [1., 2.],
        [2., 1.]])


In [155]:
_, e = torch.max(d, dim=1)
print(e)

tensor([1, 1, 0])


测试CBAM模块

In [14]:
class CBAMLayer(nn.Module):
    def __init__(self,
                 channel: int,
                 reduction=4,
                 spatial_kernel=7,
                 activation = torch.nn.ReLU,
                 scale_activation = torch.nn.Sigmoid,):
        super(CBAMLayer, self).__init__()

        # channel attention
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        # shared MLP in channel attention
        self.mlp = nn.Sequential(
            nn.Conv2d(channel, channel // reduction, 1, bias=False),    # 功能等同于全连接层
            activation(),
            nn.Conv2d(channel // reduction, channel, 1, bias=False)
        )

        # spatial attention
        self.conv = nn.Conv2d(2, 1, kernel_size=spatial_kernel, padding=spatial_kernel // 2, bias=False)
        self.scale_activation = scale_activation()

    def forward(self, x):
        # 通道注意力
        print("x:",x.size())
        max_out = self.mlp(self.max_pool(x))
        print("max_out:",max_out.size())
        avg_out = self.mlp(self.avg_pool(x))
        print("avg_out:",avg_out.size())
        channel_out = self.scale_activation(max_out + avg_out)
        print("channel_out:",channel_out.size())
        x *= channel_out
        print("x:",x.size())

        # 空间注意力
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        print("max_out:",max_out.size())
        avg_out, _ = torch.mean(x, dim=1, keepdim=True)
        print("avg_out:",avg_out.size())
        spatial_out = self.scale_activation(self.conv(torch.cat([max_out, avg_out], dim=1)))
        x *= spatial_out
        return x

In [17]:
a = torch.randn(2,4,5,5)
a.size()

torch.Size([2, 4, 5, 5])

In [16]:
cbam = CBAMLayer(4)
a = cbam(a)
a.size()

x: torch.Size([2, 4, 5, 5])
max_out: torch.Size([2, 4, 1, 1])
avg_out: torch.Size([2, 4, 1, 1])
channel_out: torch.Size([2, 4, 1, 1])
x: torch.Size([2, 4, 5, 5])
max_out: torch.Size([2, 1, 5, 5])
avg_out: torch.Size([1, 5, 5])


RuntimeError: Tensors must have same number of dimensions: got 4 and 3

In [26]:
a = torch.randn(2,4,2,2)
print(a)
avg_out = torch.mean(a, dim=1, keepdim=True)
print(avg_out)
print("avg_out:",avg_out.size())

tensor([[[[ 0.8823, -2.1015],
          [-1.5589,  0.0796]],

         [[ 0.2767, -0.1671],
          [ 2.7195, -0.5380]],

         [[-0.5586,  0.0856],
          [-1.4114, -0.1080]],

         [[ 0.0236,  0.2704],
          [-0.2852,  0.2097]]],


        [[[ 0.3468,  1.5255],
          [-0.6913,  0.5706]],

         [[ 0.0058, -0.1551],
          [-0.2402,  0.5148]],

         [[-0.5006,  0.6281],
          [-2.2064,  1.4147]],

         [[ 0.2465,  0.3509],
          [ 0.0755, -0.4617]]]])
tensor([[[[ 0.1560, -0.4782],
          [-0.1340, -0.0892]]],


        [[[ 0.0246,  0.5873],
          [-0.7656,  0.5096]]]])
avg_out: torch.Size([2, 1, 2, 2])


去掉全连接层（fc层）

In [4]:
model = models.efficientnet_v2_s()
model.classifier[1]

Linear(in_features=1280, out_features=1000, bias=True)

In [3]:
model.fc = nn.LeakyReLU(1)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
a = torch.randn(3,2)
a

tensor([[ 0.5174,  0.5121],
        [-0.5119,  0.2928],
        [-0.4977, -1.2400]])

In [5]:
ac = nn.LeakyReLU()
ac(a)

tensor([[ 0.5174,  0.5121],
        [-0.0051,  0.2928],
        [-0.0050, -0.0124]])

In [6]:
ac = nn.LeakyReLU(1)
ac(a)

tensor([[ 0.5174,  0.5121],
        [-0.5119,  0.2928],
        [-0.4977, -1.2400]])

In [7]:
# 好像成了
img = torch.randn(2,3,224,224)
out = model(img)
out.shape

torch.Size([2, 2048])

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

            
class resnet_ensembling(nn.Module):
    def __init__(
        self,
        weak_learner: Callable[..., nn.Module],
        fc_length: int = 2048,
        class_num: int = 2,
    ) -> None:
        self.model_num = len(weak_learner)
        self.models = []
        for model in weak_learner:
            model.fc = nn.LeakyReLU(1)
            set_parameter_requires_grad(model, True)
            self.models.append(model)
        self.combination = nn.Linear(model_num * fc_length, class_num)
    
    def forward(self, x: Tensor) -> Tensor:
        feature = []
        for model in self.models:
            out = model(x)
            feature.append(out)
        feature = troch.cat(feature, 1)
        feature = self.bn(feature)
        feature = self.combination(feature)
        return feature

测试encoder层

In [3]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
ln = nn.LayerNorm(256)
src = torch.rand(10, 2048)
out = encoder_layer(src)
out.shape

AssertionError: was expecting embedding dimension of 512, but got 2048

In [4]:
src = torch.rand(2, 6144)
src = src.view(2, -1, 256)
src.shape

torch.Size([2, 24, 256])

In [5]:
encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=8)
out = encoder_layer(src)
out.shape

torch.Size([2, 24, 256])

In [6]:
out = ln(out)
out.shape

torch.Size([2, 24, 256])

In [7]:
out = out.view(2,-1)
out.shape

torch.Size([2, 6144])

In [8]:
out.shape[0]

2

In [6]:
a = torch.randn(2,1,3)
b = torch.randn(2,1,5)

In [7]:
c = torch.cat((a,b),2)

In [10]:
c.size()

torch.Size([2, 1, 8])

In [2]:
model = torchvision.models.resnet101()

In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
model.layer4

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): 

In [5]:
model.classifier.in_features

1024

In [2]:
a = [1, 2, 3, 4]
b = list(range(len(a)))
b

[0, 1, 2, 3]

In [2]:
10 / 3

3.3333333333333335

In [3]:
a = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.LayerNorm(512),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 2, bias=False),
        )

In [5]:
a[-1].in_features

512

测试张量拼接

In [3]:
a = torch.randn(64,2)
b = torch.randn(64,2)
b = a + b

c = torch.cat([a,b], dim=0)
print(c.size())

torch.Size([128, 2])


In [14]:
a = torch.ones(1,1,2,8)
a

tensor([[[[1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.]]]])

In [15]:
a = a.norm(dim=-1, keepdim=True, p=2)
a

tensor([[[[2.8284],
          [2.8284]]]])

In [14]:
a = ([1,2,1,2,1,2])
b = ([1,2,3,4,5,6])

In [16]:
a[:]

[1, 2, 1, 2, 1, 2]

In [10]:
import torch.nn.functional as F

# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
print('input.shape = ', input.shape)
target = torch.randint(5, (3,), dtype=torch.int64)
print('target.shape = ', target.shape)
print(target)
loss = F.cross_entropy(input, target)
loss.backward()

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
print('input.shape = ', input.shape)
target = torch.randn(3, 5).softmax(dim=1)
print('target.shape = ', target.shape)
loss = F.cross_entropy(input, target)
loss.backward()

input.shape =  torch.Size([3, 5])
target.shape =  torch.Size([3])
tensor([1, 0, 1])
input.shape =  torch.Size([3, 5])
target.shape =  torch.Size([3, 5])


In [8]:
a = torch.randn(64)
b = torch.randn(64)
print(a.shape)
print(b)

torch.Size([64])
tensor([-0.9327,  0.1915,  0.0873, -0.0388, -1.5158, -0.9315, -0.2085, -1.7000,
         0.0559,  0.5599, -0.2764,  1.1807, -0.4583, -0.0987,  0.5731, -0.6555,
         2.8229, -0.5123,  1.5649, -0.1747,  0.2019,  0.9617,  0.9766,  0.9996,
         0.5662, -2.3061, -0.3973,  0.0641, -0.3041,  1.0716,  0.5142,  0.4007,
        -1.4754,  0.8498, -0.4011,  0.3414, -0.7452, -0.9229, -0.7684,  0.1977,
        -0.3378,  2.4418,  0.6940, -0.5529, -1.3869, -1.2504,  0.7955, -0.0593,
        -0.4837,  0.7927, -1.4871,  0.1763,  0.7193,  0.9425, -0.2407, -0.1577,
        -1.4671,  0.5832, -0.4333,  0.4288, -0.7244, -0.6726,  2.0173,  0.2645])


In [10]:
out = torch.stack((a, b), dim = 1)
print(out.shape)
print(out[0])

torch.Size([64, 2])
tensor([ 0.1849, -0.9327])


In [15]:
a = torch.randn(64)
b = torch.ones(a.shape)
print(b.shape)
print(b)

torch.Size([64])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [5]:
class MyDataloader:
    def __init__(self):
        self.count = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.count += 1
        str = '*'*(2 * self.count - 1)
        if self.count >= 3:
            raise StopIteration
        return str
        
dataloader = MyDataloader()
myiter = iter(dataloader)
# print(dir(dataloader))

print(it.__next__())

AttributeError: 'str' object has no attribute '__next__'

In [2]:
class MyNumbers:
    def __iter__(self):
        self.a = 1
        return self
 
    def __next__(self):
        if self.a <= 20:
            x = self.a
            self.a += 1
            return x
        else:
            raise StopIteration

myclass = MyNumbers()
myiter = iter(myclass)
 
for x in myiter:
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [17]:
t=torch.tensor([[7, 8, 9],[10, 11, 12]])
print(t.nelement())
idx = torch.randperm(t.nelement())
print(idx)
# t = t.view(-1)[idx]
t = t.view(-1)[idx].view(t.size())
print(t)

6
tensor([2, 1, 4, 3, 5, 0])
tensor([[ 9,  8, 11],
        [10, 12,  7]])


In [33]:
t=torch.tensor([[1, 2, 3],[3, 4, 5]])
print(t.shape)
idx = torch.randperm(t.shape[1])
t = t[:, idx].view(t.size())
print(t)

torch.Size([2, 3])
tensor([[1, 2, 3],
        [3, 4, 5]])


In [70]:
class MyDataloader:
    def __init__(self, pytorchDataloader, batchSize, shuffle=False, device='cpu'):
        # sample功能考虑以后再做
        # 要求pytorchDataloader的batchSize等于其数据量
        self.iterNum = None
        for item in pytorchDataloader:
            self.data = item
        self.data = self.data.to(device)
        self.dataNum = self.data.shape[0]
        self.batchSize = batchSize
        self.shuffle = shuffle
        

    def __iter__(self):
        self.iterNum = 0
        self.maxIterNum = self.dataNum // self.batchSize
        if self.dataNum % self.batchSize > 0:
            self.mod = 1
        else:
            self.mod = 0
        if self.shuffle:
            self.shuffleList = torch.randperm(self.dataNum)
        return self

    def __next__(self):
        if self.iterNum >= self.maxIterNum + self.mod:
            raise StopIteration
        self.iterNum += 1
        startIdx = (self.iterNum - 1) * self.batchSize
        endIdx = min(self.iterNum * self.batchSize, self.dataNum)
        if self.shuffle:
            return self.data[self.shuffleList[startIdx:endIdx]]
        else:
            return self.data[startIdx:endIdx]
    
    def initGetData(self):
        self.__iter__()
        return
    
    def getNextData(self):
        # 超出界限的数据不要了
        if self.iterNum >= self.maxIterNum:
            self.iterNum = 0
        return self.__next__()

In [71]:
t=torch.tensor([[[1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2], [3, 3, 3, 2, 2, 2], [4, 4, 4, 2, 2, 2]]])
print(t.shape)
dataLoader = MyDataloader(t, 1, shuffle=True)

dataLoader.initGetData()
for n in range(2):
    print(dataLoader.getNextData())
    
for i in dataLoader:
    print(i)

torch.Size([1, 4, 6])
tensor([[2, 2, 2, 2, 2, 2]], device='cuda:0')
tensor([[3, 3, 3, 2, 2, 2]], device='cuda:0')
tensor([[3, 3, 3, 2, 2, 2]], device='cuda:0')
tensor([[2, 2, 2, 2, 2, 2]], device='cuda:0')
tensor([[4, 4, 4, 2, 2, 2]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')


In [72]:
dataLoader = MyDataloader(torch.randn(1,642,3,64,64), 64)
for i in dataLoader:
    print(i.shape)

torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([64, 3, 64, 64])
torch.Size([2, 3, 64, 64])


In [3]:
a = torch.randn(64, 2)
_, preds = torch.max(a, 1)
preds.shape

torch.Size([64])

SCConv

In [81]:
class SRU(nn.Module):
    def __init__(self,
                 oup_channels:int, 
                 group_num:int = 16,
#                  gate_treshold:float = 0.5,
                 torch_gn:bool = True
                 ):
        super().__init__()
        
        # GroupNorm layerNorm的简化版，将通道C划分为若干组，在每组内执行layerNorm
        self.gn             = nn.GroupNorm( num_channels = oup_channels, num_groups = group_num ) if torch_gn else GroupBatchnorm2d(c_num = oup_channels, group_num = group_num)
#         self.gate_treshold  = gate_treshold
        self.sigomid        = nn.Sigmoid()
        self.advavg = nn.AdaptiveAvgPool2d(1)

    def forward(self,x):
        B = x.shape[0]
        gn_x        = self.gn(x)
        print('gn_x', gn_x.shape)
        w_gamma     = self.gn.weight/sum(self.gn.weight) # w_gamma为GroupNorm内的一个可训练参数，该数越大像素值变化越大，表明信息越丰富
#         print(self.gn.weight.shape, sum(self.gn.weight).item(), w_gamma.shape)
        w_gamma     = w_gamma.view(1,-1,1,1) # shape为1，C，1，1
        reweigts    = self.sigomid( gn_x * w_gamma )
        print('reweigts', reweigts.shape)
        # Gate
        gate_treshold = self.advavg(reweigts)
        print('gate_treshold', gate_treshold.shape)
        
        w1          = torch.where(reweigts > gate_treshold, torch.ones_like(reweigts), reweigts) # 大于门限值的设为1，否则保留原值
        w2          = torch.where(reweigts > gate_treshold, torch.zeros_like(reweigts), reweigts) # 大于门限值的设为0，否则保留原值
        print('w1', w1.shape)
        print('w2', w2.shape)
        x_1         = w1 * x
        x_2         = w2 * x
        print('x_1', w1.shape)
        print('x_2', w2.shape)
        y           = self.reconstruct(x_1,x_2)
        return y
    
    def reconstruct(self,x_1,x_2):
        x_11,x_12 = torch.split(x_1, x_1.size(1)//2, dim=1)
        x_21,x_22 = torch.split(x_2, x_2.size(1)//2, dim=1)
        return torch.cat([ x_11+x_22, x_12+x_21 ],dim=1)

In [82]:
a = torch.randn(1, 32, 14, 14)
module = SRU(32)
a = module(a)
print(a.shape)

gn_x torch.Size([1, 32, 14, 14])
reweigts torch.Size([1, 32, 14, 14])
gate_treshold torch.Size([1, 32, 1, 1])
w1 torch.Size([1, 32, 14, 14])
w2 torch.Size([1, 32, 14, 14])
x_1 torch.Size([1, 32, 14, 14])
x_2 torch.Size([1, 32, 14, 14])
torch.Size([1, 32, 14, 14])


In [32]:
class CRU(nn.Module):
    '''
    alpha: 0<alpha<1
    '''

    def __init__(self,
                 input_channel: int,
                 op_channel: int,
#                  alpha: float = 1 / 2,
#                  squeeze_radio: int = 2,
                 group_size: int = 2,
                 group_kernel_size: int = 3,
                 ):
        super().__init__()
        assert input_channel >= op_channel, '非降维'
        assert input_channel % 2 == 0, '输入通道非偶数'
        alpha = 0.5
        squeeze_radio = 2
        self.up_channel = up_channel = int(alpha * input_channel)
        self.low_channel = low_channel = input_channel - up_channel
        print(up_channel, low_channel)
        self.squeeze1 = nn.Conv2d(up_channel, up_channel // squeeze_radio, kernel_size=1, bias=False)
        self.squeeze2 = nn.Conv2d(low_channel, low_channel // squeeze_radio, kernel_size=1, bias=False)
        print(up_channel // squeeze_radio, low_channel // squeeze_radio)
        # up
        self.GWC = nn.Conv2d(up_channel // squeeze_radio, op_channel, kernel_size=group_kernel_size, stride=1,
                             padding=group_kernel_size // 2, groups=group_size)
#         self.PWC1 = nn.Conv2d(up_channel // squeeze_radio, op_channel, kernel_size=1, bias=False)
#         # low
#         self.PWC2 = nn.Conv2d(low_channel // squeeze_radio, op_channel - low_channel // squeeze_radio, kernel_size=1,
#                               bias=False)

        self.DWC = nn.Conv2d(up_channel // squeeze_radio, op_channel, kernel_size=group_kernel_size, groups=(up_channel // squeeze_radio), padding=group_kernel_size // 2)
        self.PWC = nn.Conv2d(low_channel // squeeze_radio, op_channel - low_channel // squeeze_radio, kernel_size=1, bias=False)
        print(op_channel - low_channel // squeeze_radio)
        self.advavg = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        # Split
        up, low = torch.split(x, [self.up_channel, self.low_channel], dim=1)
        up, low = self.squeeze1(up), self.squeeze2(low)
        # Transform
        Y1 = self.GWC(up) + self.DWC(up)
        Y2 = torch.cat([self.PWC(low), low], dim=1)
        # Fuse
        out = torch.cat([Y1, Y2], dim=1)
        out = functional.softmax(self.advavg(out), dim=1) * out
        
        out1, out2 = torch.split(out, out.size(1) // 2, dim=1)
        return out1 + out2

In [33]:
a = torch.randn(1, 32, 14, 14)
module = CRU(32, 32)
a = module(a)
print(a.shape)

16 16
8 8
24
tensor([-0.0923, -0.3863,  0.0358,  0.3205, -0.0901,  0.0714,  0.3079,  0.0876,
         0.2239,  0.3405,  0.1768, -0.1863, -0.1733,  0.3027,  0.4154, -0.2898,
         0.0765, -0.0379,  0.2733,  0.0724, -0.3568,  0.2389,  0.1066,  0.3414,
         0.2144,  0.3855,  0.1485,  0.1253, -0.1116, -0.1964, -0.3980, -0.1477,
        -0.0100,  0.0112,  0.0231,  0.0059,  0.0110, -0.0137,  0.0181, -0.0044,
         0.0162, -0.0067,  0.0259,  0.0243, -0.0130, -0.0726,  0.0037,  0.0213,
        -0.0060,  0.0174, -0.0509,  0.0227,  0.0331,  0.0136, -0.0220, -0.0054,
        -0.0205,  0.0564, -0.0824, -0.0462,  0.0087, -0.0628, -0.0092, -0.0393],
       grad_fn=<ReshapeAliasBackward0>)
torch.Size([1, 32, 14, 14])


In [85]:
a = torch.tensor([[[[1., 2.], [2., 2.]], [[2., 3.], [2., 2.]]]])
advavg = nn.AdaptiveAvgPool2d(1)
b = advavg(a)
print(a.shape)
print(b.shape)

out = torch.where(a > b, a, torch.zeros_like(a))
print(out)

# out = torch.where(a > c, torch.ones_like(a), -2)
# print(out)

torch.Size([1, 2, 2, 2])
torch.Size([1, 2, 1, 1])
tensor([[[[0., 2.],
          [2., 2.]],

         [[0., 3.],
          [0., 0.]]]])
